### Display a mosaic of the LSST camera

In [ ]:
import os
import numpy as np
import lsst.daf.persistence as dafPersist
import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.afw.display as afwDisplay
import matplotlib.pyplot as plt   # Not needed for image displays, but used for analysis

%matplotlib ipympl
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10, 7)

In [ ]:
afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
dataRoot = "/project/shared/BOT"
calibRoot=os.path.join(dataRoot, "rerun/rhl", "CALIB-6813D")

butler = dafPersist.Butler(dataRoot, calibRoot=calibRoot)

camera = butler.get("camera")

dataId = dict(run='12526')

It'll be easier to see if we correct for the gains.

To do this, we'll define a custom callback:

In [ ]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image, subtracting the overscan correcting for the gain"""

    return cameraGeomUtils.rawCallback(im, ccd, imageSource,
                                       subtractBias=True, correctGain=False)

In [ ]:
disp = afwDisplay.Display(1, reopenPlot=True)


disp.scale('asinh', 'zscale', Q=2)
disp.setImageColormap('viridis' if True else 'gray')

#
# Set detectorNameList to only display a subset of the detectors
#
if False:   # just a single raft
    detectorNameList = [det.getName() for det in camera if "R22" in det.getName()]
else:
    detectorNameList = None

dataType = "raw"
seqNum = 32
mos = cameraGeomUtils.showCamera(camera,
                                 cameraGeomUtils.ButlerImage(butler, dataType, run=dataId["run"], seqNum=seqNum,
                                                             callback=myCallback, verbose=True),
                                 binSize=16, detectorNameList=detectorNameList, display=disp, overlay=False,
                                 title=f'{dataId["run"]} {seqNum}')

In [ ]:
#plt.savefig("BOT-Flammarion.png")

In [ ]:
disp = afwDisplay.Display(1, reopenPlot=True)
disp.setImageColormap('viridis' if False else 'gray')

disp.scale('asinh', 'zscale', Q=2)

disp.mtv(mos)
plt.title(f'{dataId["run"]} {seqNum}');